In [1]:
import pandas as pd
import datetime as dt
import regex as re
import os
from pathlib import Path
import utils_laramie
import numpy as np

### Read in the data

In [2]:
# def get_all_raw_data(dirpath = Path('raw_data/')):
#     csv_list = [filename for filename in os.listdir(dirpath)]
#     return csv_list
# def clean_col_names(list_dfs, list_csvs):

#     list_cols = [df.columns.to_list() for df in list_dfs]
#     zipped = zip(list_csvs,list_cols)
#     name_list = []
#     column_list = []
#     for tup in list(zipped):
#         name = tup[0]
#         match = re.match(r'(\w+)_',name)
#         name = match.group(1)
#         for col in tup[1]:
#             if name.lower() != col.lower():
#                 name_list.append((name.upper()+'_'+col.lower()))
#             else:
#                 name_list.append(name.upper())

#     return name_list

# def drop_unnamed(df):
#     for col in df.columns:
#         if 'Unnamed' in str(col):
#             df.drop(columns= col, inplace=True)
#     return df
    
# def get_df(list_of_csvs= ['SPY_data.csv','TR_data.csv','VIX_data.csv']):
#     """concat a list of csvs into a single df"""
#     # make a list of dataframes
#     list_dfs = [pd.read_csv(f'raw_data/{_csv}', parse_dates = True, infer_datetime_format = True) for _csv in list_of_csvs]
#     for df in list_dfs:
#         df.drop_duplicates(inplace=True)
#         df['date'] = pd.to_datetime(df['date']).dt.date
#         df['date'] = pd.to_datetime(df['date'])
#         df = drop_unnamed(df)
#     clean_headers = clean_col_names(list_dfs,list_of_csvs)
#     merged_df = pd.concat(list_dfs, axis=1, join= 'inner')
#     merged_df.columns = clean_headers
#     return merged_df
# x = get_df()
# x 

In [3]:
# def get_df(list_of_csvs= ['SPY_data.csv','TR_data.csv','VIX_data.csv']):
#     """concat a list of csvs into a single df"""
#     list_dfs = [pd.read_csv(f'raw_data/{_csv}', parse_dates = True, infer_datetime_format = True) for _csv in list_of_csvs]
#     for df in list_dfs:
#         df.drop_duplicates(inplace=True)
#         df['date'] = pd.to_datetime(df['date']).dt.date
#         df['date'] = pd.to_datetime(df['date'])
#         df = drop_unnamed(df)
#         df.set_index('date',inplace=True)
#     clean_headers = clean_col_names(list_dfs,list_of_csvs)
#     merged_df = pd.concat(list_dfs, axis=1, join= 'inner')  
#     merged_df.columns = clean_headers
#     return merged_df
# x = get_df()
# x 

In [4]:
# imports for creating dataframe
from utils_laramie import get_df, get_all_raw_data

# imports for getting weekly range
from utils_laramie import calc_weekly_range

#imports for grouping data into weekly windows
from utils_laramie import grp_y_wk_d, drop_off_weeks




### Create Weekly Range

In [5]:
# def get_day_names(df):    
#     df = get_df(get_all_raw_data())
#     df.reset_index(inplace=True)
#     df['DayOfWeek'] = df['date'].dt.day_name()
#     df.set_index('date', inplace=True)
#     return df

# def calc_weekly_range(df):
#     """ Must run "get_day_names" first"""
#     week_high = 0  
#     week_low = 9999999
#     for index, row in df.iterrows():
#         if df.loc[index, 'DayOfWeek'] == 'Monday':
#             week_high = df.loc[index,'SPY_high']
#             week_low = df.loc[index,'SPY_low']
#         else: 
#             if df.loc[index,'SPY_high'] > week_high:
#                 week_high=df.loc[index,'SPY_high']
#             if df.loc[index,'SPY_low'] < week_low:
#                 week_low=df.loc[index,'SPY_low']
#             if df.loc[index,'DayOfWeek'] == 'Friday':
#                 df.loc[index,'weekly_range'] = week_high - week_low
#     return df.fillna(0)

In [9]:
df = (get_df(get_all_raw_data()))
df = calc_weekly_range(df)
df = grp_y_wk_d(df)
df = drop_off_weeks(df)
df.head()

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range
0,209.20,209.7295,207.20,208.27,102027111.0,374705.0,208.276128,2.5295,15.65,17.18,15.58,15.84,0.0000
1,206.48,208.2890,205.78,206.99,103372367.0,387782.0,206.966276,2.5090,17.69,18.33,16.52,17.60,0.0000
2,206.20,208.6800,204.18,205.33,162401537.0,586210.0,206.034646,4.5000,18.05,20.13,15.72,19.61,0.0000
3,205.44,207.4300,205.14,205.86,116128858.0,404992.0,206.102975,2.2900,19.25,19.72,18.13,19.34,0.0000
4,203.38,204.1400,201.51,201.88,211173305.0,669924.0,203.150102,4.3500,21.36,25.27,20.88,24.39,8.2195


In [7]:
# def drop_off_weeks(df_input):
#     idx = pd.IndexSlice
#     df_input_ind = df_input.index.to_list()
#     week_numbers = [tup[1] for tup in df_input_ind ]
#     year_numbers = set([tup[0] for tup in df_input_ind ])
#     year_numbers = sorted(list(year_numbers))
#     df_copy = df_input.copy()

#     for yr_num in year_numbers:
#         for wk_num in week_numbers:
            
#                 try:
#                     num_days = len(df_input.loc[idx[yr_num, wk_num]].index.to_list())
#                     if num_days < 5:
#                        df_copy.drop(labels = wk_num, level=1, axis=0, inplace= True)
#                 except:
#                     continue
#     df_copy = df_copy.reset_index()
#     df_copy.drop(columns=['year', 'week', 'day'], inplace= True)
#     return df_copy

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range
0,209.20,209.7295,207.20,208.27,102027111.0,374705.0,208.276128,2.5295,15.65,17.18,15.58,15.84,0.0000
1,206.48,208.2890,205.78,206.99,103372367.0,387782.0,206.966276,2.5090,17.69,18.33,16.52,17.60,0.0000
2,206.20,208.6800,204.18,205.33,162401537.0,586210.0,206.034646,4.5000,18.05,20.13,15.72,19.61,0.0000
3,205.44,207.4300,205.14,205.86,116128858.0,404992.0,206.102975,2.2900,19.25,19.72,18.13,19.34,0.0000
4,203.38,204.1400,201.51,201.88,211173305.0,669924.0,203.150102,4.3500,21.36,25.27,20.88,24.39,8.2195


In [577]:
# def calc_weekly_range(df):
#     """ Must run "get_day_names" first"""
#     week_high = 0  
#     week_low = 9999999
#     for index, row in df.iterrows():
#         if df.loc[index, 'DayOfWeek'] == 'Monday':
#             week_high = df.loc[index,'SPY_high']
#             week_low = df.loc[index,'SPY_low']
#         else: 
#             if df.loc[index,'SPY_high'] > week_high:
#                 week_high=df.loc[index,'SPY_high']
#             if df.loc[index,'SPY_low'] < week_low:
#                 week_low=df.loc[index,'SPY_low']
#             if df.loc[index,'DayOfWeek'] == 'Friday':
#                 df.loc[index,'weekly_range'] = week_high - week_low
#     return df.fillna(0)

### Testing for get_array method

In [518]:
wk_grp_ind = week_group.index.to_list()
week_numbers = [tup[1] for tup in wk_grp_ind]
year_numbers = set([tup[0] for tup in wk_grp_ind])


### get_array method

In [522]:
def get_array(df_input):
    idx = pd.IndexSlice
    df_input_ind = df_input.index.to_list()
    week_numbers = [tup[1] for tup in df_input_ind ]
    year_numbers = set([tup[0] for tup in df_input_ind ])
    year_numbers = sorted(list(year_numbers))
    
    week_data = []

    for yr_num in year_numbers:
        for wk_num in week_numbers: 
            try:
                week_data.append(df_input.loc[idx[yr_num, wk_num, :]].values)
            except:
                continue
            
    return week_data


In [523]:
def get_array_test(df_input):
    idx = pd.IndexSlice
    df_input_ind = df_input.index.to_list()
    week_numbers = [tup[1] for tup in df_input_ind ]
    year_numbers = set([tup[0] for tup in df_input_ind ])
    year_numbers = sorted(list(year_numbers))
    
    week_data = []

    for yr_num in year_numbers:
        for wk_num in week_numbers:
            
                try:
                    num_days = len(df_input_ind.loc[idx[yr_num, wk_num]].index.to_list())
                    # gets only weeks with 5 days
                    if num_days==5:
                        week_data.append(df_input.loc[idx[yr_num, wk_num, :]].values)
                except:
                    continue
            
    return week_data

### Check for Nans in Target df. Shift so that target matches features

In [525]:
# display(df_x.head(20))
# display(df_y[df_y.weekly_range != 0].head(7).shift(-1))

In [531]:
df_x.reset_index()

,year,week,day,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close
0,2015,49,3,210.60,211.0000,208.2300,208.54,108069059.0,367013.0,209.563055,2.7700,15.04,16.49,14.71,15.91
1,2015,49,4,208.90,209.1500,204.7511,205.58,166224154.0,546768.0,206.878936,4.3989,15.87,19.35,15.86,18.11
2,2015,49,5,206.10,209.9700,205.9300,209.66,192878747.0,556731.0,208.178631,4.3900,17.43,17.65,14.69,14.81
3,2015,50,1,209.20,209.7295,207.2000,208.27,102027111.0,374705.0,208.276128,2.5295,15.65,17.18,15.58,15.84
4,2015,50,2,206.48,208.2890,205.7800,206.99,103372367.0,387782.0,206.966276,2.5090,17.69,18.33,16.52,17.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,2022,12,3,446.90,448.4900,443.7100,443.80,77982935.0,627281.0,446.058683,5.8800,23.04,24.03,22.64,23.57
1588,2022,12,4,445.82,450.5000,444.7600,450.48,61411569.0,526865.0,447.609014,6.7000,23.61,23.79,21.49,21.67
1589,2022,12,5,451.09,452.9800,448.4300,452.69,77199328.0,661033.0,451.047221,4.5500,21.87,22.86,20.80,20.81
1590,2022,13,1,452.04,455.9100,450.0600,455.91,68356914.0,620868.0,453.294974,5.8500,22.14,23.33,19.54,19.63


In [532]:
def drop_off_weeks(df_input):
    idx = pd.IndexSlice
    df_input_ind = df_input.index.to_list()
    week_numbers = [tup[1] for tup in df_input_ind ]
    year_numbers = set([tup[0] for tup in df_input_ind ])
    year_numbers = sorted(list(year_numbers))
    df_copy = df_input.copy()

    for yr_num in year_numbers:
        for wk_num in week_numbers:
            
                try:
                    num_days = len(week_group.loc[idx[yr_num, wk_num]].index.to_list())
                    if num_days < 5:
                       df_copy.drop(labels = wk_num, level=1, axis=0, inplace= True)
                except:
                    continue  
    return df_copy.reset_index()

In [533]:
test_df = drop_off_weeks(week_group)

In [534]:
test_df.drop(columns=['year', 'week', 'day'], inplace= True)


In [535]:
test_df['weekly_range_test'] = test_df['weekly_range'].shift(-5)
# test_df.dropna(inplace=True)

In [536]:
len(test_df[test_df.columns.to_list()[:-2]].values)

1055

In [545]:
test_df.fillna(0, inplace=True)

In [546]:
testing_shape = np.reshape(test_df[test_df.columns.to_list()[:-2]].values, (211, 5, 12))

In [552]:
len(testing_shape[:-1])

210

In [550]:
len(test_df[test_df['weekly_range_test'] != 0 ]['weekly_range_test'])

210